In [2]:
import random
import math
from tqdm import tqdm

class BeadAgent:
    def __init__(self, alpha=0.1, gamma=0.9, eps=0.1, eps_decay=0.9995):
        self.memory_boxes = {}
        self.trace = []
        self.alpha = alpha
        self.gamma = gamma
        self.eps = eps
        self.eps_decay = eps_decay

    def choose_action(self, state):
        if state not in self.memory_boxes:
            self.memory_boxes[state] = {idx: 0 for idx in range(9) if state[idx] == '-'}

        options = self.memory_boxes[state]

        # Exploration vs exploitation
        if random.random() < self.eps:
            action = random.choice(list(options.keys()))
        else:
            action = max(options, key=options.get)

        self.trace.append((state, action))
        return action

    def apply_learning(self, outcome):
        for step in range(len(self.trace) - 1, -1, -1):
            board, move = self.trace[step]
            nxt = self.trace[step + 1][0] if step + 1 < len(self.trace) else None

            if nxt is not None:
                future_value = max(self.memory_boxes[nxt].values()) if self.memory_boxes[nxt] else 0
                reward = self._calc_reward(board, move)
                updated_q = (1 - self.alpha) * self.memory_boxes[board][move] + \
                            self.alpha * (reward + self.gamma * future_value)
            else:
                updated_q = outcome

            self.memory_boxes[board][move] = updated_q

        self.trace = []
        self.eps *= self.eps_decay

    def _calc_reward(self, board, move):
        new_state = board[:move] + 'X' + board[move + 1:]
        if is_winner(new_state):
            return 1
        elif is_winner(new_state.replace('X', 'O').replace('-', 'X')):
            return 0.5
        return 0.1


def is_winner(board):
    wins = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),
        (0, 3, 6), (1, 4, 7), (2, 5, 8),
        (0, 4, 8), (2, 4, 6)
    ]
    for a, b, c in wins:
        if board[a] == board[b] == board[c] != '-':
            return True
    return False


class OptimalBot:
    def __init__(self, depth_limit=5):
        self.depth_limit = depth_limit

    def choose_action(self, board):
        best = None
        max_val = -math.inf

        for pos in range(9):
            if board[pos] == '-':
                sim = board[:pos] + 'O' + board[pos + 1:]
                score = self._minimax(sim, 0, False)
                if score > max_val:
                    max_val = score
                    best = pos

        return best

    def _minimax(self, node, depth, maximizing):
        if is_winner(node):
            return -1 if maximizing else 1
        if '-' not in node:
            return 0

        if maximizing:
            best_score = -math.inf
            for pos in range(9):
                if node[pos] == '-':
                    nxt = node[:pos] + 'O' + node[pos + 1:]
                    best_score = max(best_score, self._minimax(nxt, depth + 1, False))
            return best_score

        else:
            best_score = math.inf
            for pos in range(9):
                if node[pos] == '-':
                    nxt = node[:pos] + 'X' + node[pos + 1:]
                    best_score = min(best_score, self._minimax(nxt, depth + 1, True))
            return best_score


def run_match(agent, enemy):
    board = "-" * 9
    turn = agent

    while True:
        move = turn.choose_action(board)
        board = board[:move] + ('X' if turn == agent else 'O') + board[move + 1:]

        if is_winner(board):
            return 1 if turn == agent else -1

        if '-' not in board:
            return 0

        turn = enemy if turn == agent else agent


def train_agent(agent, enemy, rounds):
    for _ in tqdm(range(rounds), desc="MENACE Training"):
        result = run_match(agent, enemy)
        agent.apply_learning(result)


# --- TRAINING ---

agent = BeadAgent()
enemy = OptimalBot(depth_limit=5)

print("Starting training...")
train_agent(agent, enemy, 10000)
print("\nTraining done!")

# --- PLAY AGAINST AGENT ---
print("\nPlay against the trained agent!")
current = agent
board = "-" * 9

while True:
    print(f"\nBoard:\n{board[:3]}\n{board[3:6]}\n{board[6:]}")

    if current == agent:
        mv = agent.choose_action(board)
        print(f"Agent played: {mv}")
    else:
        mv = int(input("Your move (0-8): "))
        while board[mv] != '-':
            mv = int(input("Invalid, try again: "))

    board = board[:mv] + ('X' if current == agent else 'O') + board[mv + 1:]

    if is_winner(board):
        print(f"\nFinal:\n{board[:3]}\n{board[3:6]}\n{board[6:]}")
        print("Agent wins!" if current == agent else "You win!")
        break

    if '-' not in board:
        print("\nDraw!")
        print(f"\nFinal:\n{board[:3]}\n{board[3:6]}\n{board[6:]}")
        break

    current = enemy if current == agent else agent

# Final credit assignment
if current == agent:
    agent.apply_learning(-1)
else:
    agent.apply_learning(1)


Starting training...


MENACE Training: 100%|██████████| 10000/10000 [18:04<00:00,  9.22it/s]



Training done!

Play against the trained agent!

Board:
---
---
---
Agent played: 0

Board:
X--
---
---
Your move (0-8): 4

Board:
X--
-O-
---
Agent played: 1

Board:
XX-
-O-
---
Your move (0-8): 2

Board:
XXO
-O-
---
Agent played: 6

Board:
XXO
-O-
X--
Your move (0-8): 3

Board:
XXO
OO-
X--
Agent played: 5

Board:
XXO
OOX
X--
Your move (0-8): 8

Board:
XXO
OOX
X-O
Agent played: 7

Draw!

Final:
XXO
OOX
XXO
